<a href="https://colab.research.google.com/github/yasinazhdari/CallBacks_Utilization/blob/main/a_Callback_to_cancel_training_on_75_percent_accuracy_on_Hoda_Dataset_Keras_Yasin_Azhdari_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
!mkdir dataset
!wget https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat -P dataset

--2022-06-28 05:57:40--  https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 917 [text/plain]
Saving to: ‘dataset.py’

dataset.py          100%[===================>]     917  --.-KB/s    in 0s      

2022-06-28 05:57:40 (43.7 MB/s) - ‘dataset.py’ saved [917/917]

--2022-06-28 05:57:41--  https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/Alireza-Akhavan/deeplearning-tensorflow2-notebooks/raw/master/dataset/Data_hoda

In [4]:
# 1.Import libraries                       and modules
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import numpy as np
from dataset import load_hoda
import matplotlib.pyplot as plt



In [5]:
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        print(logs)
        if(logs.get('accuracy')>0.75):
            print("\nReached 75% accuracy, Time to cancel the training!")
            self.model.stop_training = True
            
callbacks = myCallback()

In [7]:
np.random.seed(123)              # for reproducibility

# Load pre-shuffled HODA data into train and test sets
x_train_original, y_train_original, x_test_original, y_test_original = load_hoda(
                                                                        training_sample_size   =3500,
                                                                        test_sample_size=400,size=28)

# Preprocess input data

x_train = np.array(x_train_original)
x_test  = np.array( x_test_original)

x_train = x_train.astype('float32')
x_test   = x_test.astype('float32')
x_train /= 255
x_test  /= 255

# Reshape to original image shape (n x 784)  ==> (n x 28 x 28 x 1)
x_train = x_train.reshape(-1,28,28,1)
x_test  =  x_test.reshape(-1,28,28,1)


# 4. Preprocess class labels
y_train = keras.utils.to_categorical(y_train_original, num_classes=10)
y_test  = keras.utils.to_categorical(y_test_original , num_classes=10)


# test and validation set
x_val  = x_test[:200]
x_test = x_test[200:]
y_val  = y_test[:200]
y_test = y_test[200:]

# 5. Define model architecture
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

# 6. Compile model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


model.fit(x_train,                     y_train, epochs=10, callbacks=[callbacks])

Epoch 1/10
110/110 [==============================] - 1s 4ms/step - loss: 1.1772 - accuracy: 0.5951
Epoch 2/10
 99/110 [==========================>...] - ETA: 0s - loss: 0.5146 - accuracy: 0.8261{'loss': 0.4978245198726654, 'accuracy': 0.8328571319580078}

Reached 75% accuracy, Time to cancel the training!
110/110 [==============================] - 0s 4ms/step - loss: 0.4978 - accuracy: 0.8329
